In [ ]:
import warnings
warnings.filterwarnings('ignore')
from astropy.io import fits
import os
from astropy.table import join
from glob import glob 
import sys
sys.path.append('./')
from modules import dendro_dendro, dendro_misc, dendro_props, dendro_mask

In [ ]:
# Define names and filenames...

galaxy = 'ngc1433'
galaxy_hst = galaxy
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies'

hstha_file = '%s/data_hstha/%s/hst_contsub/%s_hst_ha_sic.fits' %(root_dir, galaxy_hst, galaxy_hst)
muscat_file = '%s/data_hstha/%s/muse/%s_nebmask.fits' %(root_dir, galaxy_hst, galaxy.upper())
musha_file = '%s/data_hstha/%s/muse/%s-*_MAPS.fits' %(root_dir, galaxy_hst, galaxy.upper())
musha_file = glob(musha_file)[0] #because of resolution in name

cutout_dir = '%s/data_hstha_nebulae_catalogue/%s/cutouts' %(root_dir, galaxy_hst)
dendro_dir = '%s/data_hstha_nebulae_catalogue/%s/dendro' %(root_dir, galaxy_hst)
cutouts_hdus_dir = '%s/data_hstha_nebulae_catalogue/%s/cutouts_hdus' %(root_dir, galaxy_hst)

rerun_all = False
rerun_masking = False

regions_file = '%s/sample.reg' %cutout_dir
regions_pickel_file = '%s/sample.pickel' %cutout_dir
sample_table_file = '%s/data_misc/sample_table/phangs_sample_table_v1p6.fits' %root_dir
muscat_table_file = '%s/data_misc/nebulae_catalogue/Nebulae_catalogue_v3.fits' %root_dir

print(hstha_file)
print(muscat_file)
print(musha_file)
print(cutout_dir)
print(dendro_dir)
print(cutouts_hdus_dir)
print(regions_file)
print(regions_pickel_file)
print(sample_table_file)
print(muscat_table_file)

In [ ]:
# Load regions, sample table and HDUs... 

hdus_cutouts = dendro_misc.load_pickle('%s/hdus_all.pickel' %cutout_dir)
regions = dendro_misc.load_pickle(regions_pickel_file)

sample_table = dendro_misc.get_galaxyprops(galaxy, sample_table_file)
muscat_table = dendro_misc.get_museprops(galaxy, muscat_table_file)

In [5]:
# Load cutout hdus with smoothed, masked, and non-masked data...

hdus_file = '%s/hdus_all_withmasked.pickel' %cutout_dir
muscat_regionIDs_file =  '%s/muscat_regionIDs.pickel' %cutout_dir

if os.path.exists(hdus_file) & ~rerun_masking:
    muscat_regionIDs = muscat_table['region_ID']
    hdus = dendro_misc.load_pickle(hdus_file)
else: 
    muscat_regionIDs = muscat_table['region_ID']
    hdus = dendro_dendro.get_maskedhdus(hdus_cutouts, regions, muscat_regionIDs)
    dendro_misc.save_pickle(hdus, hdus_file)

[INFO] [load_pickle] Load /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1433/cutouts/hdus_all_withmasked.pickel


In [52]:
i = 1
output = dendro_dendro.get_dedro(hdus['hstha_hdu_smooth_masked'][i],
                                hdus['hstha_hdu_smooth_maskedall'][i], 
                                hdus['hstha_hdu_smooth'][i],
                                sampletable=sample_table,
                                min_npix=9, 
                                min_value_sig=3, 
                                min_delta_sig=3)
dendro, props, indexmap_trunk_hdu, indexmap_trunk_close_hdu = output

props is not None

False

In [46]:
indexmap_trunk_hdu.data[indexmap_trunk_hdu.data!=-1] = 100
indexmap_trunk_close_hdu.data[indexmap_trunk_close_hdu.data!=-1] = 100
plt.imshow(indexmap_trunk_hdu.data)

muscat_regionIDs[1]

AttributeError: 'NoneType' object has no attribute 'data'

In [44]:
from matplotlib import pyplot as plt
import numpy as np

print(hdus.keys())
print(np.unique(hdus['musmask_hdu'][1].data))
# plt.imshow(hdus['musmask_hdu'][1].data)

dict_keys(['hstha_hdu', 'musha_hdu', 'muscat_hdu', 'hstha_hdu_smooth', 'hstha_hdu_smooth_masked', 'hstha_hdu_smooth_masked_ones', 'hstha_hdu_smooth_maskedall', 'musmask_hdu'])
[0]


In [54]:
# Get dendrogram for all... 

from importlib import reload
reload(dendro_dendro)

props_all, hdus = dendro_dendro.get_dedro_all(hdus, sample_table, muscat_regionIDs)

Dendrogram:   0%|          | 0/1736 [00:00<?, ?it/s]

[INFO] [get_dedro] [regionID=1.0] No dendro found...
[INFO] [get_dedro] Running dummy dendro...
[INFO] [get_dedro] [regionID=2.0] No dendro found...
[INFO] [get_dedro] [regionID=3.0] No dendro found...
[INFO] [get_dedro] [regionID=4.0] No dendro found...
[INFO] [get_dedro] [regionID=8.0] No dendro found...
[INFO] [get_dedro] [regionID=12.0] No dendro found...
[INFO] [get_dedro] [regionID=13.0] No dendro found...
[INFO] [get_dedro] [regionID=19.0] No dendro found...
[INFO] [get_dedro] [regionID=21.0] No dendro found...
[INFO] [get_dedro] [regionID=22.0] No dendro found...
[INFO] [get_dedro] [regionID=25.0] No dendro found...
[INFO] [get_dedro] [regionID=26.0] No dendro found...
[INFO] [get_dedro] [regionID=29.0] No dendro found...
[INFO] [get_dedro] [regionID=30.0] No dendro found...
[INFO] [get_dedro] [regionID=32.0] No dendro found...
[INFO] [get_dedro] [regionID=35.0] No dendro found...
[INFO] [get_dedro] [regionID=36.0] No dendro found...
[INFO] [get_dedro] [regionID=37.0] No dendro

In [55]:
# Add additional infomation from MUSE and save... 

props_all_muscat_table = join(props_all, muscat_table, keys='region_ID')

props_all_muscat_table['flux_corr'] = dendro_props.correct_ha_flux(props_all_muscat_table)
props_all_muscat_table['ha_lum_hst'] = dendro_props.calculate_luminosity(props_all_muscat_table['flux_corr']*1e-20, sample_table['dist'].quantity[0])
    
props_all_muscat_table.rename_column('flux', 'HA6562_FLUX_HST')
props_all_muscat_table.rename_column('flux_corr', 'HA6562_FLUX_HST_CORR')
props_all_muscat_table.rename_column('ha_lum_hst', 'HA6562_LUMINOSITY_HST')

dendro_misc.save_pickle(hdus, '%s/hdus_all_withmasked_withdendro.pickel' %dendro_dir)
dendro_misc.save_pickle(props_all_muscat_table, '%s/props_all.pickel' %dendro_dir)        
props_all_muscat_table.write('%s/props_all.fits' %dendro_dir, overwrite=True)

[INFO] [save_pickle] Saved to /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1433/dendro/hdus_all_withmasked_withdendro.pickel
[INFO] [save_pickle] Saved to /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1433/dendro/props_all.pickel


In [57]:
# Creat masks and region files... 

# Save HDUs as HDU lists for each region with RegionID
dendro_dendro.get_hdulists(hdus, muscat_regionIDs, outputdir=cutouts_hdus_dir)

# Save region file of masks 
dendro_mask.get_ds9regions_circ_decor(props_all, '%s/%s_trunk_regions.reg' %(dendro_dir, galaxy))
dendro_mask.get_ds9regions_ellip_decor(props_all, '%s/%s_sigma_regions.reg' %(dendro_dir, galaxy))

# Save the mask for whole mosaic
hstha_hdu = fits.open(hstha_file)[0]
dendro_mask.get_hdumask(hstha_hdu, hdus['indexmap_trunk_hdu'], '%s/%s_trunk_mask.fits'  %(dendro_dir, galaxy))
# dendro_mask.get_hdumask(hstha_hdu, hdus['indexmap_trunk_close_hdu'], '%s/%s_trunk_close_mask.fits'  %(dendro_dir, galaxy))

  0%|          | 0/1736 [00:00<?, ?it/s]

[INFO] [get_ds9regions_circ_decor] Saved regions to /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1433/dendro/ngc1433_trunk_regions.reg
[INFO] [get_ds9regions_ellip_decor] Saved regions to /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc1433/dendro/ngc1433_sigma_regions.reg


Masking regions:   0%|          | 0/1736 [00:00<?, ?it/s]